In [1]:
import process_engine as PE
import process_object as PO

In [9]:
# define objects
att_user = [
    PO.ObjectAttribute(att_name='username', att_dtype='string', att_required=True),
    PO.ObjectAttribute(att_name='usertype', att_dtype='string', att_required=True)
]
o_user = PO.ProcessObjectType(o_name='user', o_attributes=att_user)
o_user_instance = PO.ProcessObjectInstance(o_type=o_user, attributes={'username': 'u1', 'usertype': 'A', 'sex': 'W'})

att_material = [
    PO.ObjectAttribute(att_name='name', att_dtype='string', att_required=True),
    PO.ObjectAttribute(att_name='unit', att_dtype='string', att_required=True),
    PO.ObjectAttribute(att_name='color', att_dtype='string', att_required=False)
]
o_material = PO.ProcessObjectType(o_name='material', o_attributes=att_material)
o_material_instance = PO.ProcessObjectInstance(o_type=o_material, attributes={'name': 'mat1', 'unit': 'kg'})

In [11]:
o_material_instance.o_id

UUID('b55fe982-e8da-4561-ad0f-11bc2bfafa49')